In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

os.path.exists('/content/drive/MyDrive/Capstone_GE_DSI_CV_Project/preprocessed_data/CT_lung_cancer/PETCT_2dac5ef654_axial_277.jpg')

False

In [ ]:
!pip install pydicom
!pip install opencv-python
!pip install pillow # optional
!pip install pandas
!pip3 install numpy
!pip3 install dicom2nifti
!pip3 install nibabel
!pip3 install pydicom
!pip3 install tqdm
!pip3 install nilearn
!pip install --quiet torchio==0.18.90

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.7/172.7 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 9.5 MB/s eta 0:00:00


In [ ]:
import pathlib as plb
import tempfile
import os
import dicom2nifti
import nibabel as nib
import numpy as np
import pydicom
import sys
import shutil
import nilearn.image
from tqdm import tqdm

import enum
import time
import random
import multiprocessing
from pathlib import Path

import torch
import torchvision
import torchio as tio
import torch.nn.functional as F
import torchvision.transforms as T
from torchvision.utils import save_image

import numpy as np
# from unet import UNet
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt

from IPython import display
from tqdm.auto import tqdm
from pathlib import Path

from PIL import Image

from skimage.measure import regionprops_table

In [ ]:
train_num_samples = 5000
val_num_samples = 2000
test_num_samples = 1000

In [ ]:
def get_randomized_subset(arr, num_samples, seed):
  np.random.seed(seed)
  neg_slices_index = np.random.choice(len(arr), num_samples, replace=False)
  neg_file_names = np.array(arr)[neg_slices_index]

  return neg_file_names

In [ ]:
path = '/content/drive/MyDrive/Capstone_GE_DSI_CV_Project/preprocessed_data/negative/data'
all_neg_files = os.listdir(path)
print(len(all_neg_files))

FileNotFoundError: ignored

In [ ]:
from numpy.random import default_rng
import random

num_samples = train_num_samples + val_num_samples + test_num_samples
seed = 10

neg_file_names = get_randomized_subset(all_neg_files, num_samples, seed)

In [ ]:
cancer_samples = pd.read_csv('/content/drive/MyDrive/Capstone_GE_DSI_CV_Project/preprocessed_data/CT_labels.csv')
lung_cancer_df, lymphoma_df, melanoma_df = cancer_samples[cancer_samples.cancer_type=='lung_cancer'], cancer_samples[cancer_samples.cancer_type=='lymphoma'], cancer_samples[cancer_samples.cancer_type=='melanoma']

print(len(lung_cancer_df), len(lymphoma_df), len(melanoma_df))

In [ ]:
# LUNG CANCER
all_lung_files = lung_cancer_df.img_filename.tolist()
num_samples = train_num_samples + val_num_samples + test_num_samples
seed = 50

lung_cancer_file_names = get_randomized_subset(all_lung_files, num_samples, seed)

In [ ]:
# LYMPHOMA
all_lymphoma_files = lymphoma_df.img_filename.tolist()
num_samples = train_num_samples + val_num_samples + test_num_samples
seed = 320

lymphoma_file_names = get_randomized_subset(all_lymphoma_files, num_samples, seed)

In [ ]:
# MELANOMA
all_melanoma_files = melanoma_df.img_filename.tolist()
num_samples = train_num_samples + val_num_samples + test_num_samples
seed = 921

melanoma_file_names = get_randomized_subset(all_melanoma_files, num_samples, seed)

In [ ]:
def train_val_test_split(arr, seed):

  split1 = int(train_num_samples/num_samples * len(arr))
  split2 = int((train_num_samples+val_num_samples)/num_samples * len(arr))

  np.random.seed(seed)
  np.random.shuffle(arr)
  train = arr[:split1]
  val = arr[split1:split2]
  test = arr[split2:]
  return train, val, test

In [ ]:
seed = 10
lung_train, lung_val, lung_test = train_val_test_split(lung_cancer_file_names, seed)

In [ ]:
seed = 10
lymphoma_train, lymphoma_val, lymphoma_test = train_val_test_split(lymphoma_file_names, seed)

In [ ]:
seed = 10
melanoma_train, melanoma_val, melanoma_test = train_val_test_split(melanoma_file_names, seed)

In [ ]:
seed = 10
negative_train, negative_val, negative_test = train_val_test_split(neg_file_names, seed)

In [ ]:
lung_train = np.array([lung_train, ['lung_cancer' for i in range(len(lung_train))], ['train' for i in range(len(lung_train))]]).T

In [ ]:
lymphoma_train = np.array([lymphoma_train, ['lymphoma' for i in range(len(lymphoma_train))], ['train' for i in range(len(lymphoma_train))]]).T

In [ ]:
melanoma_train = np.array([melanoma_train, ['melanoma' for i in range(len(melanoma_train))], ['train' for i in range(len(melanoma_train))]]).T

In [ ]:
negative_train = np.array([negative_train, ['negative' for i in range(len(negative_train))], ['train' for i in range(len(negative_train))]]).T

In [ ]:
lung_val = np.array([lung_val, ['lung_cancer' for i in range(len(lung_val))], ['val' for i in range(len(lung_val))]]).T

In [ ]:
lung_test = np.array([lung_test, ['lung_cancer' for i in range(len(lung_test))], ['test' for i in range(len(lung_test))]]).T

In [ ]:
lymphoma_val = np.array([lymphoma_val, ['lymphoma' for i in range(len(lymphoma_val))], ['val' for i in range(len(lymphoma_val))]]).T

In [ ]:
lymphoma_test = np.array([lymphoma_test, ['lymphoma' for i in range(len(lymphoma_test))], ['test' for i in range(len(lymphoma_test))]]).T

In [ ]:
melanoma_val = np.array([melanoma_val, ['melanoma' for i in range(len(melanoma_val))], ['val' for i in range(len(melanoma_val))]]).T

In [ ]:
melanoma_test = np.array([melanoma_test, ['melanoma' for i in range(len(melanoma_test))], ['test' for i in range(len(melanoma_test))]]).T

In [ ]:
negative_val = np.array([negative_val, ['negative' for i in range(len(negative_val))], ['val' for i in range(len(negative_val))]]).T

In [ ]:
negative_test = np.array([negative_test, ['negative' for i in range(len(negative_test))], ['test' for i in range(len(negative_test))]]).T

In [ ]:
full_master_list = np.concatenate((lung_train, lung_val, lung_test, lymphoma_train, lymphoma_val, lymphoma_test, melanoma_train, melanoma_val, melanoma_test, negative_train, negative_val, negative_test))

In [ ]:
full_master_list_df = pd.DataFrame(full_master_list, columns = ['file_name','cancer_type','split'])
full_master_list_df

In [ ]:
print(len(full_master_list_df[full_master_list_df['split']=='train']), len(full_master_list_df[full_master_list_df['split']=='val']), len(full_master_list_df[full_master_list_df['split']=='test']))

In [ ]:
full_master_list_df.to_csv('/content/drive/MyDrive/Capstone_GE_DSI_CV_Project/preprocessed_data/train_test_split_master.csv', index=False)

In [ ]:
df = full_master_list_df[full_master_list_df["file_name"].duplicated()]
df

In [ ]:
for f in df.file_name.unique():
  for i in ['train','test','val']:
    folder = f'/content/drive/MyDrive/Capstone_GE_DSI_CV_Project/preprocessed_data/CT_DETR/{i}'
    if os.path.exists(folder+f'/{f}'):
      os.remove(folder+f'/{f}')

    folder = f'/content/drive/MyDrive/Capstone_GE_DSI_CV_Project/preprocessed_data/SUV_DETR/{i}'
    if os.path.exists(folder+f'/{f}'):
      os.remove(folder+f'/{f}')

In [ ]:
full_master_list_v2 = full_master_list_df[~full_master_list_df.file_name.isin(df.file_name.unique())]

In [ ]:
full_master_list_v2.to_csv('/content/drive/MyDrive/Capstone_GE_DSI_CV_Project/preprocessed_data/train_test_split_master_v2.csv', index=False)

In [ ]:
df.file_name.unique()

In [ ]:
file = open('/content/drive/MyDrive/Capstone_GE_DSI_CV_Project/preprocessed_data/leaked.txt','w')

for item in df.file_name.unique():
  file.write('gs://petct_preprocessed_data/SUV_DETR/'+item+"\n")
  file.write('gs://petct_preprocessed_data/CT_DETR/'+item+"\n")
file.close()